In [1]:

import os
import sys
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from statsmodels.tsa.stattools import adfuller

# ----------------------------
# Configuration
# ----------------------------
TICKERS = ["TSLA", "BND", "SPY"]
START_DATE = "2015-07-01"
END_DATE = "2025-07-31"
ROLL_WINDOW = 30  # days for rolling statistics
OUT_DIR = "outputs"
PLOTS_DIR = os.path.join(OUT_DIR, "plots")
np.random.seed(42)

os.makedirs(PLOTS_DIR, exist_ok=True)

# ----------------------------
# Utilities
# ----------------------------
def download_data(tickers, start, end):
    print(f"Downloading {tickers} from {start} to {end} ...")
    data = {}
    for t in tickers:
        df = yf.download(t, start=start, end=end, progress=False)
        if df.empty:
            print(f"Warning: no data downloaded for {t}.")
        # Ensure Date is index as DatetimeIndex
        if not isinstance(df.index, pd.DatetimeIndex):
            df.index = pd.to_datetime(df.index)
        data[t] = df
    return data

def fill_missing(df):
    # Forward then backward fill and drop remaining NaNs if any
    df = df.copy()
    df = df.ffill().bfill()
    df = df.dropna(how="all")
    return df

def compute_features(df):
    df = df.copy()
    # Adjusted close exists in Yahoo; prefer 'Adj Close' for returns if present
    price_col = "Adj Close" if "Adj Close" in df.columns else "Close"
    df["Price"] = df[price_col]
    df["Daily_Return"] = df["Price"].pct_change()
    df["Return_Z"] = (df["Daily_Return"] - df["Daily_Return"].mean()) / df["Daily_Return"].std()
    df[f"Rolling_Mean_{ROLL_WINDOW}"] = df["Price"].rolling(window=ROLL_WINDOW).mean()
    df[f"Rolling_Std_{ROLL_WINDOW}"] = df["Price"].rolling(window=ROLL_WINDOW).std()
    return df

def adf_test(series, title="Series"):
    series = series.dropna()
    if series.shape[0] < 10:
        return {"ADF Statistic": np.nan, "p-value": np.nan, "usedlags": np.nan, "nobs": series.shape[0], "stationary": None}
    res = adfuller(series, autolag="AIC")
    out = {
        "ADF Statistic": res[0],
        "p-value": res[1],
        "usedlags": res[2],
        "nobs": res[3],
        "stationary": res[1] <= 0.05
    }
    return out

def compute_risk_metrics(returns):
    returns = returns.dropna()
    if returns.empty:
        return {"mean": np.nan, "std": np.nan, "VaR_95": np.nan, "Sharpe_annual": np.nan}
    mean = returns.mean()
    std = returns.std()
    VaR_95 = np.percentile(returns, 5)  # 5th percentile
    sharpe = (mean / std) * np.sqrt(252) if std != 0 else np.nan
    return {"mean": mean, "std": std, "VaR_95": VaR_95, "Sharpe_annual": sharpe}

def save_df(df, path):
    df.to_csv(path)
    print(f"Saved: {path}")

# ----------------------------
# Main workflow
# ----------------------------
def main():
    start_time = datetime.now()
    # 1) Download
    data = download_data(TICKERS, START_DATE, END_DATE)

    summary_metrics = {}
    for ticker, df in data.items():
        print("\n" + "="*60)
        print(f"Processing ticker: {ticker}")
        if df.empty:
            print(f"Skipping {ticker} because no data.")
            continue

        # 2) Clean / fill missing
        df = fill_missing(df)

        # 3) Compute features
        df = compute_features(df)

        # 4) EDA quick stats
        print(f"\n{ticker} - head:")
        print(df.head(3).to_string())

        print(f"\n{ticker} - describe (Price):")
        print(df["Price"].describe().to_string())

        # 5) Stationarity tests
        adf_price = adf_test(df["Price"].dropna(), title=f"{ticker} Price")
        adf_returns = adf_test(df["Daily_Return"].dropna(), title=f"{ticker} Daily_Return")
        print(f"\nADF test for {ticker} price: ADF={adf_price['ADF Statistic']:.4f}, p={adf_price['p-value']:.4f}, stationary={adf_price['stationary']}")
        print(f"ADF test for {ticker} returns: ADF={adf_returns['ADF Statistic']:.4f}, p={adf_returns['p-value']:.4f}, stationary={adf_returns['stationary']}")

        # 6) Outlier detection (z-score > 3)
        outliers = df[np.abs(df["Return_Z"]) > 3]
        print(f"\n{ticker} - outliers (|z|>3) count: {len(outliers)}")
        if not outliers.empty:
            print(outliers[["Price", "Daily_Return", "Return_Z"]].head(5).to_string())

        # 7) Risk metrics
        metrics = compute_risk_metrics(df["Daily_Return"])
        metrics.update({
            "ADF_price_pvalue": adf_price["p-value"],
            "ADF_returns_pvalue": adf_returns["p-value"],
            "outliers_count": len(outliers),
            "start_date": df.index.min().strftime("%Y-%m-%d"),
            "end_date": df.index.max().strftime("%Y-%m-%d")
        })
        summary_metrics[ticker] = metrics

        # 8) Save csv
        save_df(df, os.path.join(OUT_DIR, f"{ticker}_processed.csv"))

        # 9) Plots
        # a) Price over time with rolling mean/std band
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.plot(df.index, df["Price"], label=f"{ticker} Price")
        ax.plot(df.index, df[f"Rolling_Mean_{ROLL_WINDOW}"], label=f"{ROLL_WINDOW}-day Rolling Mean")
        upper = df[f"Rolling_Mean_{ROLL_WINDOW}"] + df[f"Rolling_Std_{ROLL_WINDOW}"]
        lower = df[f"Rolling_Mean_{ROLL_WINDOW}"] - df[f"Rolling_Std_{ROLL_WINDOW}"]
        ax.fill_between(df.index, lower, upper, alpha=0.15)
        ax.set_title(f"{ticker} Price with {ROLL_WINDOW}-day rolling mean & std")
        ax.set_xlabel("Date")
        ax.set_ylabel("Price (USD)")
        ax.legend()
        fig.tight_layout()
        ppath = os.path.join(PLOTS_DIR, f"{ticker}_price_rolling.png")
        fig.savefig(ppath, dpi=150)
        plt.close(fig)
        print(f"Saved plot: {ppath}")

        # b) Daily returns plot
        fig, ax = plt.subplots(figsize=(12, 4))
        ax.plot(df.index, df["Daily_Return"], label="Daily Return")
        ax.set_title(f"{ticker} Daily Returns")
        ax.set_xlabel("Date")
        ax.set_ylabel("Daily Return")
        fig.tight_layout()
        rpath = os.path.join(PLOTS_DIR, f"{ticker}_daily_returns.png")
        fig.savefig(rpath, dpi=150)
        plt.close(fig)
        print(f"Saved plot: {rpath}")

        # c) Histogram of returns with VaR line
        fig, ax = plt.subplots(figsize=(8, 4))
        ax.hist(df["Daily_Return"].dropna(), bins=100)
        ax.axvline(metrics["VaR_95"], linestyle="--", linewidth=2, label=f"VaR 95% = {metrics['VaR_95']:.4f}")
        ax.set_title(f"{ticker} Returns Distribution")
        ax.set_xlabel("Daily Return")
        ax.set_ylabel("Frequency")
        ax.legend()
        hpath = os.path.join(PLOTS_DIR, f"{ticker}_returns_hist.png")
        fig.tight_layout()
        fig.savefig(hpath, dpi=150)
        plt.close(fig)
        print(f"Saved plot: {hpath}")

    # 10) Summary table
    summary_df = pd.DataFrame.from_dict(summary_metrics, orient="index")
    if not summary_df.empty:
        summary_df = summary_df[["start_date","end_date","mean","std","VaR_95","Sharpe_annual","ADF_price_pvalue","ADF_returns_pvalue","outliers_count"]]
        save_df(summary_df, os.path.join(OUT_DIR, "summary_metrics.csv"))
        print("\nSummary metrics (head):")
        print(summary_df.head().to_string())
    else:
        print("No summary metrics to save (no data).")

    elapsed = datetime.now() - start_time
    print(f"\nCompleted Task 1 in {elapsed}. Outputs saved in '{OUT_DIR}' directory.")

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print("Error during execution:", str(e))
        sys.exit(1)


C:\Users\Amenzz\AppData\Local\Temp\ipykernel_4284\769387050.py:30: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(t, start=start, end=end, progress=False)
C:\Users\Amenzz\AppData\Local\Temp\ipykernel_4284\769387050.py:30: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(t, start=start, end=end, progress=False)
C:\Users\Amenzz\AppData\Local\Temp\ipykernel_4284\769387050.py:30: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(t, start=start, end=end, progress=False)



Processing ticker: TSLA

TSLA - head:
Price           Close       High        Low       Open     Volume      Price Daily_Return  Return_Z Rolling_Mean_30 Rolling_Std_30
Ticker           TSLA       TSLA       TSLA       TSLA       TSLA                                                                 
Date                                                                                                                              
2015-07-01  17.943333  18.174667  17.856667  18.073999   31518000  17.943333          NaN       NaN             NaN            NaN
2015-07-02  18.667999  18.830000  18.220667  18.680000  107458500  18.667999     0.040386  1.034139             NaN            NaN
2015-07-06  18.648001  18.779333  18.420000  18.591999   61828500  18.648001    -0.001071 -0.077763             NaN            NaN

TSLA - describe (Price):
count    2535.000000
mean      131.963002
std       120.914904
min         9.578000
25%        18.967667
50%        94.571335
75%       236.761665
ma

In [6]:


import os
import warnings
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt

# ARIMA
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima.model import ARIMA

warnings.filterwarnings("ignore")
np.random.seed(42)
tf.random.set_seed(42)

# -------------------------
# Configuration
# -------------------------
TICKER = "TSLA"
START_DATE = "2015-07-01"
END_DATE = "2025-07-31"
TRAIN_END = "2023-12-31"   # inclusive train end
TEST_START = "2024-01-01"
OUT_DIR = "outputs_task2"
PLOTS_DIR = os.path.join(OUT_DIR, "plots")
os.makedirs(PLOTS_DIR, exist_ok=True)

# -------------------------
# Utilities
# -------------------------
def download_tsla(start, end):
    df = yf.download(TICKER, start=start, end=end, progress=False)
    if df.empty:
        raise RuntimeError("No data downloaded. Check ticker/date range or internet.")
    if "Adj Close" in df.columns:
        df["Price"] = df["Adj Close"]
    else:
        df["Price"] = df["Close"]
    df = df[["Price"]]
    df.index = pd.to_datetime(df.index)
    return df

def split_train_test(df, train_end):
    train = df.loc[:train_end].copy()
    test = df.loc[train_end:].copy()
    # ensure test begins next trading day after train_end; we will slice properly by TEST_START below
    return train, test

def evaluate_series(y_true, y_pred):
    # y_true, y_pred as 1D arrays aligned
    mae = mean_absolute_error(y_true, y_pred)
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / np.where(y_true==0, 1e-8, y_true))) * 100
    return {"MAE": mae, "RMSE": rmse, "MAPE%": mape}
def train_forecast_arima(train_series, n_periods):
    print("Fitting ARIMA model (statsmodels)...")
    # You can manually tune (p,d,q) or try a few combinations
    p, d, q = 5, 1, 0  # Example order
    model = ARIMA(train_series, order=(p, d, q))
    fitted = model.fit()

    forecast_res = fitted.get_forecast(steps=n_periods)
    fc = forecast_res.predicted_mean
    confint = forecast_res.conf_int()

    print("ARIMA order used:", (p, d, q))
    return fc.values, confint.values, fitted

# -------------------------
# LSTM Model
# -------------------------
def create_sequences(series_values, window):
    X, y = [], []
    for i in range(window, len(series_values)):
        X.append(series_values[i-window:i])
        y.append(series_values[i])
    X = np.array(X)
    y = np.array(y)
    # reshape X for LSTM: (samples, timesteps, features)
    return X.reshape((X.shape[0], X.shape[1], 1)), y

def train_lstm_forecast(train_series, test_len, window=60, epochs=25, batch_size=32):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(train_series.values.reshape(-1, 1)).flatten()

    # prepare supervised sequences
    X_train, y_train = create_sequences(scaled, window)
    if X_train.size == 0:
        raise RuntimeError("Not enough data for LSTM with the chosen window size.")
    
    model = Sequential()
    model.add(LSTM(64, input_shape=(X_train.shape[1], 1), return_sequences=False))
    model.add(Dropout(0.1))
    model.add(Dense(1))
    model.compile(optimizer="adam", loss="mse")

    # Train
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    # Forecast iteratively
    last_window = scaled[-window:].tolist()
    preds_scaled = []
    for _ in range(test_len):
        x_in = np.array(last_window[-window:]).reshape((1, window, 1))
        yhat = model.predict(x_in, verbose=0)[0][0]
        preds_scaled.append(yhat)
        last_window.append(yhat)  # append predicted scaled value

    preds = scaler.inverse_transform(np.array(preds_scaled).reshape(-1, 1)).flatten()
    return preds, model, scaler
def run_task2():
    print("Downloading data...")
    df = download_tsla(START_DATE, END_DATE)
    # enforce TEST_START slicing
    train_df = df.loc[:TRAIN_END].copy()
    test_df = df.loc[TEST_START:END_DATE].copy()
    if test_df.empty:
        raise RuntimeError("Test set is empty. Check TEST_START and END_DATE.")
    print(f"Data downloaded: total {len(df)} rows, train {len(train_df)}, test {len(test_df)}")

    # ---------- ARIMA ----------
    arima_horizon = len(test_df)
    arima_fc, arima_confint, arima_model = train_forecast_arima(train_df["Price"], arima_horizon)
    # align ARIMA forecast index with test_df index
    arima_index = test_df.index[:arima_horizon]
    arima_series = pd.Series(arima_fc, index=arima_index, name="ARIMA_Forecast")
    arima_conf = pd.DataFrame(arima_confint, index=arima_index, columns=["lower", "upper"])

    # ---------- LSTM ----------
    # Use only train data to fit scaler and model; then forecast test_len ahead
    lstm_preds, lstm_model, lstm_scaler = train_lstm_forecast(train_df["Price"], test_len=arima_horizon, window=60, epochs=30, batch_size=32)
    lstm_series = pd.Series(lstm_preds, index=arima_index, name="LSTM_Forecast")

    # ---------- Evaluate ----------
    actual = test_df["Price"].loc[arima_index]
    arima_metrics = evaluate_series(actual.values, arima_series.values)
    lstm_metrics = evaluate_series(actual.values, lstm_series.values)

    # Save results
    results_df = pd.DataFrame({
        "Actual": actual,
        "ARIMA_Forecast": arima_series,
        "LSTM_Forecast": lstm_series,
        "ARIMA_lower": arima_conf["lower"],
        "ARIMA_upper": arima_conf["upper"]
    })
    results_path = os.path.join(OUT_DIR, "tsla_forecasts.csv")
    results_df.to_csv(results_path)
    print(f"Saved forecasts: {results_path}")

    # ---------- Plots ----------
    plt.figure(figsize=(12,6))
    plt.plot(train_df.index[-500:], train_df["Price"].iloc[-500:], label="Train (last 500 days)")
    plt.plot(actual.index, actual.values, label="Actual (Test)", linewidth=1.2)
    plt.plot(arima_series.index, arima_series.values, label="ARIMA Forecast", linestyle="--")
    plt.plot(lstm_series.index, lstm_series.values, label="LSTM Forecast", linestyle=":")
    plt.fill_between(arima_conf.index, arima_conf["lower"], arima_conf["upper"], color="gray", alpha=0.2, label="ARIMA 95% CI")
    plt.title(f"{TICKER} Forecasts: ARIMA vs LSTM")
    plt.xlabel("Date")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(PLOTS_DIR, "tsla_forecasts_compare.png"), dpi=150)
    plt.close()

    # Error plots
    plt.figure(figsize=(10,4))
    plt.plot(actual.index, (actual.values - arima_series.values), label="ARIMA Error")
    plt.plot(actual.index, (actual.values - lstm_series.values), label="LSTM Error")
    plt.title("Forecast Errors (Actual - Predicted)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(PLOTS_DIR, "tsla_forecast_errors.png"), dpi=150)
    plt.close()

    # ---------- Summary ----------
    summary = {
        "ARIMA_metrics": arima_metrics,
        "LSTM_metrics": lstm_metrics,
        "ARIMA_order": getattr(arima_model, "order", None),
        "train_start": train_df.index.min().strftime("%Y-%m-%d"),
        "train_end": train_df.index.max().strftime("%Y-%m-%d"),
        "test_start": actual.index.min().strftime("%Y-%m-%d"),
        "test_end": actual.index.max().strftime("%Y-%m-%d"),
        "n_test_days": len(actual)
    }
    summary_path = os.path.join(OUT_DIR, "summary.json")
    import json
    with open(summary_path, "w") as f:
        json.dump(summary, f, indent=2)
    print(f"Saved summary: {summary_path}")

    # Print human-readable summary
    print("\n=== Model evaluation on test set ===")
    print(f"Test period: {summary['test_start']} -> {summary['test_end']} ({summary['n_test_days']} trading days)")
    print("\nARIMA metrics:")
    for k,v in arima_metrics.items():
        print(f"  {k}: {v:.6f}")
    print("\nLSTM metrics:")
    for k,v in lstm_metrics.items():
        print(f"  {k}: {v:.6f}")

    # Which performed better?
    def better(a,b,metric="RMSE"):
        return "ARIMA" if a[metric] < b[metric] else "LSTM"
    better_rmse = better(arima_metrics, lstm_metrics, "RMSE")
    print(f"\nBetter model by RMSE: {better_rmse}")

    print("\nAll outputs (CSV, plots, summary) saved in:", OUT_DIR)
    return summary, results_df

if __name__ == "__main__":
    start = datetime.now()
    summary, results_df = run_task2()
    print("\nCompleted in:", datetime.now() - start)


Data downloaded: total 2535 rows, train 2140, test 395
Fitting ARIMA model (statsmodels)...
ARIMA order used: (5, 1, 0)
Saved forecasts: outputs_task2\tsla_forecasts.csv
Saved summary: outputs_task2\summary.json

=== Model evaluation on test set ===
Test period: 2024-01-02 -> 2025-07-30 (395 trading days)

ARIMA metrics:
  MAE: 62.968664
  RMSE: 77.985447
  MAPE%: 24.076288

LSTM metrics:
  MAE: 64.401362
  RMSE: 80.611215
  MAPE%: 24.055137

Better model by RMSE: ARIMA

All outputs (CSV, plots, summary) saved in: outputs_task2

Completed in: 0:01:14.042683


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
from statsmodels.tsa.arima.model import ARIMA

def train_forecast_arima(train_series, n_periods, order=(5,1,0)):
    model = ARIMA(train_series, order=order)
    fitted = model.fit()
    forecast_res = fitted.get_forecast(steps=n_periods)
    forecast = forecast_res.predicted_mean
    conf_int = forecast_res.conf_int()
    return forecast, conf_int, fitted

def evaluate_forecast(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    return mae, rmse

def plot_forecasts(train_df, test_df, arima_fc, arima_confint):
    plt.figure(figsize=(12, 6))
    plt.plot(train_df.index, train_df['Price'], label='Train', color='blue')
    plt.plot(test_df.index, test_df['Price'], label='Test', color='orange')
    plt.plot(test_df.index, arima_fc, label='ARIMA Forecast', color='green')
    plt.fill_between(test_df.index,
                     arima_confint.iloc[:, 0],
                     arima_confint.iloc[:, 1],
                     color='k', alpha=0.1)
    plt.legend()
    plt.title('ARIMA Forecast vs Actual')
    plt.show()

def run_task2():
    # Load data
    df = pd.read_csv("brent_daily.csv", parse_dates=['Date'])
    df = df.sort_values("Date").set_index("Date")
    df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
    df = df.dropna()

    # Train-test split (80/20)
    split_index = int(len(df) * 0.8)
    train_df = df.iloc[:split_index]
    test_df = df.iloc[split_index:]

    # ARIMA forecast
    arima_horizon = len(test_df)
    arima_order = (5, 1, 0)  # You can adjust
    arima_fc, arima_confint, arima_model = train_forecast_arima(train_df['Price'], arima_horizon, order=arima_order)

    # Evaluation
    mae, rmse = evaluate_forecast(test_df['Price'], arima_fc)

    # Results summary
    results_df = pd.DataFrame({
        'Actual': test_df['Price'],
        'Forecast': arima_fc
    }, index=test_df.index)

    plot_forecasts(train_df, test_df, arima_fc, arima_confint)

    summary = {
        "ARIMA Order": arima_order,
        "MAE": mae,
        "RMSE": rmse
    }
    return summary, results_df

if __name__ == "__main__":
    start = datetime.now()
    summary, results_df = run_task2()
    print("\nSummary Metrics:")
    for k, v in summary.items():
        print(f"{k}: {v}")
    print("\nResults head:")
    print(results_df.head())
    print("\nCompleted in:", datetime.now() - start)
